# Generate images from text phrases with VQGAN and CLIP (z + quantize method with augmentations).

Current version slightly modified from [this](https://colab.research.google.com/drive/1ud6KJeKdq5egQx_zz2-rni5R-Q-vxJdj?usp=sharing) notebook by [@angremlin](https://twitter.com/angremlin), which was based on the original VQGAN+CLIP [notebook](https://colab.research.google.com/drive/1go6YwMFe5MX6XM9tv-cnQiSTU50N9EeT?fbclid=IwAR30ZqxIJG0-2wDukRydFA3jU5OpLHrlC_Sg1iRXqmoTkEhaJtHdRi6H7AI#scrollTo=FhhdWrSxQhwg) by Katherine Crowson ([Github](https://github.com/crowsonkb), [Twitter](https://twitter.com/RiversHaveWings)).

If you need help or would like to learn more, see Crowson's [help file](https://docs.google.com/document/d/1Lu7XPRKlNhBQjcKr8k8qRzUzbBW7kzxb5Vu72GMRn2E/edit#), this [Spanish help page](https://tuscriaturas.miraheze.org/wiki/Ayuda:Crear_imágenes_con_VQGAN+CLIP), and/or Unicole's [Data Prophecy how-to video](https://youtu.be/1YXo1HAIOFo).

I just added troubleshooting cells, more models to use (some hosted on my Google Drive), and some options for parameters and video generation.

--Downy Thornapple

In [ ]:
# @title Licensed under the MIT License

# Copyright (c) 2021 Katherine Crowson

# Permission is hereby granted, free of charge, to any person obtaining a copy
# of this software and associated documentation files (the "Software"), to deal
# in the Software without restriction, including without limitation the rights
# to use, copy, modify, merge, publish, distribute, sublicense, and/or sell
# copies of the Software, and to permit persons to whom the Software is
# furnished to do so, subject to the following conditions:

# The above copyright notice and this permission notice shall be included in
# all copies or substantial portions of the Software.

# THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR
# IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY,
# FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL THE
# AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER
# LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM,
# OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER DEALINGS IN
# THE SOFTWARE.

In [ ]:
# @title Optional GPU check
# See help file here for explanation: https://medium.com/analytics-vidhya/explained-output-of-nvidia-smi-utility-fc4fbee3b124

!nvidia-smi

In [ ]:
# @title Optional RAM check

from psutil import virtual_memory
ram_gb = virtual_memory().total / 1e9
print('Your runtime has {:.1f} gigabytes of available RAM\n'.format(ram_gb))

if ram_gb < 20:
  print('To enable a high-RAM runtime, select the Runtime > "Change runtime type"')
  print('menu, and then select High-RAM in the Runtime shape dropdown. Then, ')
  print('re-execute this cell.')
else:
  print('You are using a high-RAM runtime!')

In [ ]:
# @title Installing the libraries
# @markdown This cell will take a while because you have to download multiple libraries
 
print("Downloading CLIP...")
!git clone https://github.com/openai/CLIP                 &> /dev/null
 
print("Installing Python libraries for AI...")
!git clone https://github.com/CompVis/taming-transformers &> /dev/null
!pip install ftfy regex tqdm omegaconf pytorch-lightning  &> /dev/null
!pip install kornia                                       &> /dev/null
!pip install einops                                       &> /dev/null
!pip install transformers                                 &> /dev/null
 
print("Installing libraries for metadata management ...")
!pip install stegano                                      &> /dev/null
!apt install exempi                                       &> /dev/null
!pip install python-xmp-toolkit                           &> /dev/null
!pip install imgtag                                       &> /dev/null
!pip install pillow==7.1.2                                &> /dev/null
 
print("Installing python libraries for creating video ...")
!pip install imageio-ffmpeg &> /dev/null
!mkdir steps
print("Installation finished.")

In [ ]:
# @title Loading libraries and definitions

import argparse
import math
from pathlib import Path
import sys
import hashlib
from IPython.display import Audio, clear_output

sys.path.append('./taming-transformers')
from IPython import display
from base64 import b64encode
from omegaconf import OmegaConf
from PIL import Image
from taming.models import cond_transformer, vqgan
import torch
from torch import nn, optim
from torch.nn import functional as F
from torchvision import transforms
from torchvision.transforms import functional as TF
from tqdm.notebook import tqdm

from CLIP import clip
import kornia.augmentation as K
import numpy as np
import imageio
from PIL import ImageFile, Image
from imgtag import ImgTag    # metadatos 
from libxmp import *         # metadatos
import libxmp                # metadatos
from stegano import lsb
import json
import re
ImageFile.LOAD_TRUNCATED_IMAGES = True
 
def sinc(x):
    return torch.where(x != 0, torch.sin(math.pi * x) / (math.pi * x), x.new_ones([]))

def lanczos(x, a):
    cond = torch.logical_and(-a < x, x < a)
    out = torch.where(cond, sinc(x) * sinc(x/a), x.new_zeros([]))
    return out / out.sum()

def ramp(ratio, width):
    n = math.ceil(width / ratio + 1)
    out = torch.empty([n])
    cur = 0
    for i in range(out.shape[0]):
        out[i] = cur
        cur += ratio
    return torch.cat([-out[1:].flip([0]), out])[1:-1]

def resample(input, size, align_corners=True):
    n, c, h, w = input.shape
    dh, dw = size

    input = input.view([n * c, 1, h, w])

    if dh < h:
        kernel_h = lanczos(ramp(dh / h, 2), 2).to(input.device, input.dtype)
        pad_h = (kernel_h.shape[0] - 1) // 2
        input = F.pad(input, (0, 0, pad_h, pad_h), 'reflect')
        input = F.conv2d(input, kernel_h[None, None, :, None])

    if dw < w:
        kernel_w = lanczos(ramp(dw / w, 2), 2).to(input.device, input.dtype)
        pad_w = (kernel_w.shape[0] - 1) // 2
        input = F.pad(input, (pad_w, pad_w, 0, 0), 'reflect')
        input = F.conv2d(input, kernel_w[None, None, None, :])

    input = input.view([n, c, h, w])
    return F.interpolate(input, size, mode='bicubic', align_corners=align_corners)

class ReplaceGrad(torch.autograd.Function):
    @staticmethod
    def forward(ctx, x_forward, x_backward):
        ctx.shape = x_backward.shape
        return x_forward
 
    @staticmethod
    def backward(ctx, grad_in):
        return None, grad_in.sum_to_size(ctx.shape)
 
 
replace_grad = ReplaceGrad.apply
 
 
class ClampWithGrad(torch.autograd.Function):
    @staticmethod
    def forward(ctx, input, min, max):
        ctx.min = min
        ctx.max = max
        ctx.save_for_backward(input)
        return input.clamp(min, max)
 
    @staticmethod
    def backward(ctx, grad_in):
        input, = ctx.saved_tensors
        return grad_in * (grad_in * (input - input.clamp(ctx.min, ctx.max)) >= 0), None, None
 
 
clamp_with_grad = ClampWithGrad.apply
 
 
def vector_quantize(x, codebook):
    d = x.pow(2).sum(dim=-1, keepdim=True) + codebook.pow(2).sum(dim=1) - 2 * x @ codebook.T
    indices = d.argmin(-1)
    x_q = F.one_hot(indices, codebook.shape[0]).to(d.dtype) @ codebook
    return replace_grad(x_q, x)
 
 
class Prompt(nn.Module):
    def __init__(self, embed, weight=1., stop=float('-inf')):
        super().__init__()
        self.register_buffer('embed', embed)
        self.register_buffer('weight', torch.as_tensor(weight))
        self.register_buffer('stop', torch.as_tensor(stop))
 
    def forward(self, input):
        input_normed = F.normalize(input.unsqueeze(1), dim=2)
        embed_normed = F.normalize(self.embed.unsqueeze(0), dim=2)
        dists = input_normed.sub(embed_normed).norm(dim=2).div(2).arcsin().pow(2).mul(2)
        dists = dists * self.weight.sign()
        return self.weight.abs() * replace_grad(dists, torch.maximum(dists, self.stop)).mean()
 
 
def parse_prompt(prompt):
    vals = prompt.rsplit(':', 2)
    vals = vals + ['', '1', '-inf'][len(vals):]
    return vals[0], float(vals[1]), float(vals[2])
 
 
class MakeCutouts(nn.Module):
    def __init__(self, cut_size, cutn, cut_pow=1.):
        super().__init__()
        self.cut_size = cut_size
        self.cutn = cutn
        self.cut_pow = cut_pow
        self.augs = nn.Sequential(
            K.RandomHorizontalFlip(p=0.5),
            # K.RandomSolarize(0.01, 0.01, p=0.7),
            K.RandomSharpness(0.3,p=0.4),
            K.RandomAffine(degrees=30, translate=0.1, p=0.8, padding_mode='border'),
            K.RandomPerspective(0.2,p=0.4),
            K.ColorJitter(hue=0.01, saturation=0.01, p=0.7))
        self.noise_fac = 0.1
 
 
    def forward(self, input):
        sideY, sideX = input.shape[2:4]
        max_size = min(sideX, sideY)
        min_size = min(sideX, sideY, self.cut_size)
        cutouts = []
        for _ in range(self.cutn):
            size = int(torch.rand([])**self.cut_pow * (max_size - min_size) + min_size)
            offsetx = torch.randint(0, sideX - size + 1, ())
            offsety = torch.randint(0, sideY - size + 1, ())
            cutout = input[:, :, offsety:offsety + size, offsetx:offsetx + size]
            cutouts.append(resample(cutout, (self.cut_size, self.cut_size)))
        batch = self.augs(torch.cat(cutouts, dim=0))
        if self.noise_fac:
            facs = batch.new_empty([self.cutn, 1, 1, 1]).uniform_(0, self.noise_fac)
            batch = batch + facs * torch.randn_like(batch)
        return batch
 
 
def load_vqgan_model(config_path, checkpoint_path):
    config = OmegaConf.load(config_path)
    if config.model.target == 'taming.models.vqgan.VQModel':
        model = vqgan.VQModel(**config.model.params)
        model.eval().requires_grad_(False)
        model.init_from_ckpt(checkpoint_path)

    elif config.model.target == 'taming.models.vqgan.GumbelVQ':
        model = vqgan.GumbelVQ(**config.model.params)
        model.eval().requires_grad_(False)
        model.init_from_ckpt(checkpoint_path)

    elif config.model.target == 'taming.models.cond_transformer.Net2NetTransformer':
        parent_model = cond_transformer.Net2NetTransformer(**config.model.params)
        parent_model.eval().requires_grad_(False)
        parent_model.init_from_ckpt(checkpoint_path)
        model = parent_model.first_stage_model
    else:
        raise ValueError(f'unknown model type: {config.model.target}')
    del model.loss
    return model

def resize_image(image, out_size):
    ratio = image.size[0] / image.size[1]
    area = min(image.size[0] * image.size[1], out_size[0] * out_size[1])
    size = round((area * ratio)**0.5), round((area / ratio)**0.5)
    return image.resize(size, Image.LANCZOS)

def download_img(img_url):
    try:
        return wget.download(img_url,out="input.jpg")
    except:
        return

In [ ]:
#@title Selection of models to download
#@markdown By default, the notebook downloads Model 16384 from ImageNet. There are others such as ImageNet 1024, COCO-Stuff, WikiArt 1024, WikiArt 16384, FacesHQ or S-FLCKR, which are not downloaded by default, since it would be in vain if you are not going to use them, so if you want to use them, simply select the models to download.

#This step is required even if you don't download a model. Just uncheck all the boxes.

#@markdown ![Models](https://i.imgur.com/PEPqhpC.png)

imagenet_1024 = False #@param {type:"boolean"}
imagenet_16384 = False #@param {type:"boolean"}
gumbel_8192 = True #@param {type:"boolean"}
coco = False #@param {type:"boolean"}
faceshq = False #@param {type:"boolean"}
wikiart_1024 = False #@param {type:"boolean"}
wikiart_16384 = False #@param {type:"boolean"}
sflckr = False #@param {type:"boolean"}

#Below all added with last update. Results weren't very good.
ade20k = False #@param {type:"boolean"}
ffhq = False #@param {type:"boolean"}
celebahq = False #@param {type:"boolean"}

if imagenet_1024:
  !curl -L -o vqgan_imagenet_f16_1024.yaml -C - 'https://heibox.uni-heidelberg.de/d/8088892a516d4e3baf92/files/?p=%2Fconfigs%2Fmodel.yaml&dl=1' #ImageNet 1024
  !curl -L -o vqgan_imagenet_f16_1024.ckpt -C - 'https://heibox.uni-heidelberg.de/d/8088892a516d4e3baf92/files/?p=%2Fckpts%2Flast.ckpt&dl=1'  #ImageNet 1024
if imagenet_16384:
  !curl -L -o vqgan_imagenet_f16_16384.yaml -C - 'https://heibox.uni-heidelberg.de/d/a7530b09fed84f80a887/files/?p=%2Fconfigs%2Fmodel.yaml&dl=1' #ImageNet 16384
#  !curl -L -o vqgan_imagenet_f16_16384.ckpt -C - 'https://heibox.uni-heidelberg.de/d/a7530b09fed84f80a887/files/?p=%2Fckpts%2Flast.ckpt&dl=1' #ImageNet 16384
  !gdown https://drive.google.com/uc?id=135CJAtmKvhyoqJEYK8EIdveNspj4qjdc #imagenet_16384

#You can find more models at https://github.com/CompVis/taming-transformers
#It might cause a bug if this is left checked while the parameter is set to another model.
if gumbel_8192:
  !curl -L -o gumbel_8192.yaml -C - 'https://heibox.uni-heidelberg.de/d/2e5662443a6b4307b470/files/?p=%2Fconfigs%2Fmodel.yaml&dl=1' #Gumbel 8192
  !curl -L -o gumbel_8192.ckpt -C - 'https://heibox.uni-heidelberg.de/d/2e5662443a6b4307b470/files/?p=%2Fckpts%2Flast.ckpt&dl=1' #Gumbel 8192

if coco:
  !curl -L -o coco.yaml -C - 'https://dl.nmkd.de/ai/clip/coco/coco.yaml' #COCO
  !curl -L -o coco.ckpt -C - 'https://dl.nmkd.de/ai/clip/coco/coco.ckpt' #COCO
if faceshq:
  !curl -L -o faceshq.yaml -C - 'https://drive.google.com/uc?export=download&id=1fHwGx_hnBtC8nsq7hesJvs-Klv-P0gzT' #FacesHQ
  !curl -L -o faceshq.ckpt -C - 'https://app.koofr.net/content/links/a04deec9-0c59-4673-8b37-3d696fe63a5d/files/get/last.ckpt?path=%2F2020-11-13T21-41-45_faceshq_transformer%2Fcheckpoints%2Flast.ckpt' #FacesHQ
if wikiart_1024: 
  !curl -L -o wikiart_1024.yaml -C - 'http://dl.nmkd.de/ai/clip/wikiart-vqgan/WikiArt_augmented_Steps_7mil_finetuned_480k.yaml' #WikiArt 1024
#This host is slow and unstable, so I have it downloading from my Google Drive instead.
#  !curl -L -o wikiart_1024.ckpt -C - 'http://dl.nmkd.de/ai/clip/wikiart-vqgan/WikiArt_augmented_Steps_7mil_finetuned_480k.ckpt' #WikiArt 1024
  !gdown https://drive.google.com/uc?id=1X_iw1XP02B8q_Fz2s5qR5L-auyuoXQnt

if wikiart_16384:
  !curl -L -o wikiart_16384.yaml -C - 'http://eaidata.bmk.sh/data/Wikiart_16384/wikiart_f16_16384_8145600.yaml' #WikiArt 16384
#  !curl -L -o wikiart_16384.ckpt -C - 'http://eaidata.bmk.sh/data/Wikiart_16384/wikiart_f16_16384_8145600.ckpt' #WikiArt 16384
  !gdown https://drive.google.com/uc?id=1-B7tbvJpuEQJ0zjpoKVjOmNagbEdlORg #WikiArt 16384
if sflckr:
  !curl -L -o sflckr.yaml -C - 'https://heibox.uni-heidelberg.de/d/73487ab6e5314cb5adba/files/?p=%2Fconfigs%2F2020-11-09T13-31-51-project.yaml&dl=1' #S-FLCKR
  !curl -L -o sflckr.ckpt -C - 'https://heibox.uni-heidelberg.de/d/73487ab6e5314cb5adba/files/?p=%2Fcheckpoints%2Flast.ckpt&dl=1' #S-FLCKR

#Below all added with last update. The results weren't very good.
if ade20k:
  !curl -L -o ade20k.yaml -C - 'https://static.miraheze.org/intercriaturaswiki/b/bf/Ade20k.txt' #ADE20K
#  !curl -L -o ade20k.ckpt -C - 'https://app.koofr.net/content/links/0f65c2cd-7102-4550-a2bd-07fd383aac9e/files/get/last.ckpt?path=%2F2020-11-20T21-45-44_ade20k_transformer%2Fcheckpoints%2Flast.ckpt' #ADE20K
  !gdown https://drive.google.com/uc?id=17njzwDGWOjlFwd0HZBzhylTDbaxiFc1C #ADE20K
if ffhq:
  !curl -L -o ffhq.yaml -C - 'https://app.koofr.net/content/links/0fc005bf-3dca-4079-9d40-cdf38d42cd7a/files/get/2021-04-23T18-19-01-project.yaml?path=%2F2021-04-23T18-19-01_ffhq_transformer%2Fconfigs%2F2021-04-23T18-19-01-project.yaml&force' #FFHQ
#  !curl -L -o ffhq.ckpt -C - 'https://app.koofr.net/content/links/0fc005bf-3dca-4079-9d40-cdf38d42cd7a/files/get/last.ckpt?path=%2F2021-04-23T18-19-01_ffhq_transformer%2Fcheckpoints%2Flast.ckpt&force' #FFHQ
  !gdown https://drive.google.com/uc?id=1b5uJRxwvUiXugjHKwoQrSCuI2gXf4-Fy #FFHQ
if celebahq:
  !curl -L -o celebahq.yaml -C - 'https://app.koofr.net/content/links/6dddf083-40c8-470a-9360-a9dab2a94e96/files/get/2021-04-23T18-11-19-project.yaml?path=%2F2021-04-23T18-11-19_celebahq_transformer%2Fconfigs%2F2021-04-23T18-11-19-project.yaml&force' #CelebA-HQ
#  !curl -L -o celebahq.ckpt -C - 'https://app.koofr.net/content/links/6dddf083-40c8-470a-9360-a9dab2a94e96/files/get/last.ckpt?path=%2F2021-04-23T18-11-19_celebahq_transformer%2Fcheckpoints%2Flast.ckpt&force' #CelebA-HQ
  !gdown https://drive.google.com/uc?id=1RB58pPji-VDuDmWuMNwIExU7V3u_0VDI #CelebA-HQ

In [ ]:
#@markdown To use your drive, set `root_path` to the relative drive folder path you want outputs to be saved to, then execute the cell. Leaving the field blank or just not running this will have outputs save to the runtime temp storage.
root_path = "" #@param {type: "string"}
abs_root_path = "/content"
if len(root_path) > 0:
    abs_root_path = abs_root_path + "/drive/MyDrive/" + root_path

from google.colab import drive
drive.mount('/content/drive')

## Tools for execution:
Mainly what you will have to modify will be `texts:`, there you can place the textprompt(s) you want to generate (separated with `|`). It is a list because you can put more than one text, and so the AI tries to 'mix' the images, giving the same priority to both texts.

To use an initial image to the model, you just have to upload a file to the Colab environment (in the section on the left), and then modify `init_image:` putting the exact name of the file. Example: `sample.png`

You can also modify the model by changing the line that says `model:`. Currently 1024, 16384, WikiArt, S-FLCKR and COCO-Stuff are available. To activate them you have to have downloaded them first, and then you can simply select it.

You can also use `target_images`, which is basically putting one or more images on it that the AI will take as a "target", fulfilling the same function as putting text on it. To put more than one you have to use `|` as a separator.

In [ ]:
prompts = "a photorealistic image of a stained glass window | trending on artstation | unreal engine | logo:0 | U:0 | text:0" #@param {type:"string"}
  # Use percent or parts for weight, e.g., "unicole unicron:2|unreal engine:1|raytracing:1"
  # Use a weight of zero for elements to NOT include in image, e.g., "red:0"
width =  614#@param {type:"number"}
height =  432#@param {type:"number"}
model_id = "gumbel_8192" #@param ["vqgan_imagenet_f16_1024", "vqgan_imagenet_f16_16384", "gumbel_8192", "coco", "faceshq", "wikiart_1024", "wikiart_16384", "sflckr", "ade20k", "ffhq", "celebahq"]
image_interval =  50#@param {type:"number"}
init_image = "/content/intersubjectivity.PNG"#@param {type:"string"}
target_images = None#@param {type:"string"}
#@markdown
seed_single = "-1"#@param {type:"string"}
seed_list = "1,2,3" #@param {type:"string"}
seed_type = seed_single #@param ["seed_single", "seed_list"] {type:"raw"}
#@markdown
max_iterations = 501#@param {type:"number"}
run_batch = True#@param {type:"boolean"}
batch_seed_count = 10#@param {type:"number"}
input_images = ""

#@markdown Optional "advanced" options:
step_size = 0.1 #@param {type:"slider", min:0, max:1, step:0.05}
	  # A lower step size (default 0.1) or "learning rate" should be more stable.
cutn = 64#@param {type:"number"}
  	# Number of "cuts" (default 64) higher is more detailed
cut_pow = 1.0#@param {type:"number"}
		# Not sure what this does (default 1.0)
#cut_size = 1#@param {type:"number"}
		# No idea what this does (default 1)
init_weight = 0.1#@param {type:"number"}
		# No idea what this does (default 0.1)
#This will crop and resize between intervals for a "zoom" animation. It doesn't work yet.
zoom = False#@param {type:"boolean"}

model_names={"vqgan_imagenet_f16_16384": 'ImageNet 16384',"vqgan_imagenet_f16_1024":"ImageNet 1024", 
                 "wikiart_1024":"WikiArt 1024", "wikiart_16384":"WikiArt 16384", "coco":"COCO-Stuff", 
                 "gumbel_8192":"Gumbel 8192","faceshq":"FacesHQ", "sflckr":"S-FLCKR",
              	 "ade20k":"ADE20K","celebahq":"CelebA-HQ","ffhq":"Flickr-Faces-HQ"}
model_name = model_names[model_id]

if not 'abs_root_path' in globals():
    abs_root_path = "/content"

if init_image == "None":
    init_image = None
if target_images == "None" or not target_images:
    target_images = []
else:
    target_images = target_images.split("|")
    target_images = [image.strip() for image in target_images]

if init_image or target_images != []:
    input_images = True

prompts_str = prompts
prompt_hash = hashlib.md5(prompts.encode())
prompt_md5 = prompt_hash.hexdigest()

prompts = [frase.strip() for frase in prompts.split("|")]
if prompts == ['']:
    prompts = []

if seed_type == seed_single:
    seed = None if seed_single == -1 else seed_single  
else:
    seed_list = ''.join(seed_list)
    seed = seed_list.replace(" ", "").split(',')

args = argparse.Namespace(
    prompts=prompts,
    image_prompts=target_images,
    noise_prompt_seeds=[],
    noise_prompt_weights=[],
    size=[width, height],
    init_image=init_image,
    init_weight=0.,
    clip_model='ViT-B/32',
    vqgan_config=f'{model_id}.yaml',
    vqgan_checkpoint=f'{model_id}.ckpt',
    step_size=step_size,
    cutn=64,
    cut_pow=1.,
    display_freq=image_interval,
    seed=seed,
    batch_seed_count=(batch_seed_count if run_batch else 1),
    hash = prompt_md5,
    prompt_str = prompts_str,
)

print("Prompt Hash: ", args.hash)

!mkdir {abs_root_path}/{args.hash}
!rm -f {abs_root_path}/{args.hash}/prompt.txt
!echo "{prompts_str}" > {abs_root_path}/{args.hash}/prompt.txt

In [ ]:
#@title Load the model

#@markdown This cell must be run after parameters are configured, but does NOT need to be run again when parameters are changed

#@markdown Once this has been run successfully you only need to run parameters and then the program to execute with new parameters

device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
model = load_vqgan_model(args.vqgan_config, args.vqgan_checkpoint).to(device)
perceptor = clip.load(args.clip_model, jit=False)[0].eval().requires_grad_(False).to(device)

cut_size = perceptor.visual.input_resolution

if gumbel_8192:
    e_dim = model.quantize.embedding_dim
else:
    e_dim = model.quantize.e_dim

f = 2**(model.decoder.num_resolutions - 1)
make_cutouts = MakeCutouts(cut_size, args.cutn, cut_pow=args.cut_pow)

toksX, toksY = args.size[0] // f, args.size[1] // f
sideX, sideY = toksX * f, toksY * f

if gumbel_8192:
    e_dim = 256
    n_toks = model.quantize.n_embed
    z_min = model.quantize.embed.weight.min(dim=0).values[None, :, None, None]
    z_max = model.quantize.embed.weight.max(dim=0).values[None, :, None, None]
else:
  	e_dim = model.quantize.e_dim
  	n_toks = model.quantize.n_e
  	z_min = model.quantize.embedding.weight.min(dim=0).values[None, :, None, None]
  	z_max = model.quantize.embedding.weight.max(dim=0).values[None, :, None, None]

normalize = transforms.Normalize(mean=[0.48145466, 0.4578275, 0.40821073],
                                std=[0.26862954, 0.26130258, 0.27577711])

In [ ]:
#@title Do the execution

def synth(z):
    if gumbel_8192:
        z_q = vector_quantize(z.movedim(1, 3), model.quantize.embed.weight).movedim(3, 1)
    else:
        z_q = vector_quantize(z.movedim(1, 3), model.quantize.embedding.weight).movedim(3, 1)

    return clamp_with_grad(model.decode(z_q).add(1).div(2), 0, 1)

def add_xmp_data(nombrefichero):
    imagen = ImgTag(filename=nombrefichero)
    imagen.xmp.append_array_item(libxmp.consts.XMP_NS_DC, 'creator', 'VQGAN+CLIP', {"prop_array_is_ordered":True, "prop_value_is_array":True})
    if args.prompts:
        imagen.xmp.append_array_item(libxmp.consts.XMP_NS_DC, 'title', " | ".join(args.prompts), {"prop_array_is_ordered":True, "prop_value_is_array":True})
    else:
        imagen.xmp.append_array_item(libxmp.consts.XMP_NS_DC, 'title', 'None', {"prop_array_is_ordered":True, "prop_value_is_array":True})
    imagen.xmp.append_array_item(libxmp.consts.XMP_NS_DC, 'i', str(i), {"prop_array_is_ordered":True, "prop_value_is_array":True})
    imagen.xmp.append_array_item(libxmp.consts.XMP_NS_DC, 'model', model_name, {"prop_array_is_ordered":True, "prop_value_is_array":True})
    imagen.xmp.append_array_item(libxmp.consts.XMP_NS_DC, 'seed',str(seed) , {"prop_array_is_ordered":True, "prop_value_is_array":True})
    imagen.xmp.append_array_item(libxmp.consts.XMP_NS_DC, 'input_images',str(input_images) , {"prop_array_is_ordered":True, "prop_value_is_array":True})
    #for frases in args.prompts:
    #    imagen.xmp.append_array_item(libxmp.consts.XMP_NS_DC, 'Prompt' ,frases, {"prop_array_is_ordered":True, "prop_value_is_array":True})
    imagen.close()

def add_stegano_data(filename):
    data = {
        "title": " | ".join(args.prompts) if args.prompts else None,
        "notebook": "VQGAN+CLIP",
        "i": i,
        "model": model_name,
        "seed": str(seed),
        "input_images": input_images
    }
    lsb.hide(filename, json.dumps(data)).save(filename)

@torch.no_grad()
def checkin(i, losses):
    losses_str = ', '.join(f'{loss.item():g}' for loss in losses)
    tqdm.write(f'i: {i}, loss: {sum(losses).item():g}, losses: {losses_str}')
    out = synth(z)
    TF.to_pil_image(out[0].cpu()).save('progress.png')
    #no one is looking at this info just save some cycles
    #add_stegano_data('progress.png')
    #add_xmp_data('progress.png')
    display.display(display.Image('progress.png'))

def ascend_txt():
    global i
    global z

    out = synth(z)
    iii = perceptor.encode_image(normalize(make_cutouts(out))).float()

    result = []

    if args.init_weight:
        result.append(F.mse_loss(z, z_orig) * args.init_weight / 2)

    for prompt in pMs:
        result.append(prompt(iii))
    img = np.array(out.mul(255).clamp(0, 255)[0].cpu().detach().numpy().astype(np.uint8))[:,:,:]
    img = np.transpose(img, (1, 2, 0))
    filename = f"steps/{i:04}.png"
    imageio.imwrite(filename, np.array(img))
    add_stegano_data(filename)
    add_xmp_data(filename)
    return result

def train(i):
    global opt
    global z

    opt.zero_grad()
    lossAll = ascend_txt()
    if i % args.display_freq == 0:
        checkin(i, lossAll)
    loss = sum(lossAll)
    loss.backward()
    opt.step()
    with torch.no_grad():
        z.copy_(z.maximum(z_min).minimum(z_max))

def make_final_name(s):
    stepstr = "_s{s}".format(s=str(int(args.step_size*100)).zfill(3))
    noiterstr = "{path}/{hash}/{seed}{step}_".format(path=abs_root_path, hash=args.hash, seed=str(s), step=stepstr)
    expr = noiterstr + "(?P<iter>\\d{4})[.]png"
    print(expr)
    newiter = max_iterations
    if(args.init_image is not None):
        if(re.fullmatch(expr, args.init_image) is not None):
            m = re.match(expr, args.init_image)
            iter = m.group("iter")
            inum = int(iter)
            newiter = inum+newiter
    return noiterstr+str(str(newiter).zfill(4))+".png"

def save_final(s):
    out = synth(z)
    img = np.array(out.mul(255).clamp(0, 255)[0].cpu().detach().numpy().astype(np.uint8))[:,:,:]
    img = np.transpose(img, (1, 2, 0))
    filename = make_final_name(s)
    imageio.imwrite(filename, np.array(img))
    add_stegano_data(filename)
    add_xmp_data(filename)

def run_seed():
    torch.cuda.empty_cache()
    global opt
    global z
    global i

    if args.init_image:
        pil_image = Image.open(args.init_image).convert('RGB')
        pil_image = pil_image.resize((sideX, sideY), Image.LANCZOS)
        z, *_ = model.encode(TF.to_tensor(pil_image).to(device).unsqueeze(0) * 2 - 1)
    else:
        one_hot = F.one_hot(torch.randint(n_toks, [toksY * toksX], device=device), n_toks).float()

        if gumbel_8192:
          z = one_hot @ model.quantize.embed.weight
        else:
          z = one_hot @ model.quantize.embedding.weight

        z = z.view([-1, toksY, toksX, e_dim]).permute(0, 3, 1, 2)
    z_orig = z.clone()
    z.requires_grad_(True)
    opt = optim.Adam([z], lr=args.step_size)
    i = 0
    with tqdm() as pbar:
        while True:
            train(i)
            if i == max_iterations:
                break
            i += 1
            pbar.update()

print('Using device:', device)
if prompts:
    print('Using texts:', prompts)

if target_images:
    print('Using image prompts:', target_images)

if args.batch_seed_count is None:
    batch_seed_count = 10
else:
    batch_seed_count = args.batch_seed_count

pMs = []

for prompt in args.prompts:
    txt, weight, stop = parse_prompt(prompt)
    embed = perceptor.encode_text(clip.tokenize(txt).to(device)).float()
    pMs.append(Prompt(embed, weight, stop).to(device))

for prompt in args.image_prompts:
    path, weight, stop = parse_prompt(prompt)
    img = resize_image(Image.open(path).convert('RGB'), (sideX, sideY))
    batch = make_cutouts(TF.to_tensor(img).unsqueeze(0).to(device))
    embed = perceptor.encode_image(normalize(batch)).float()
    pMs.append(Prompt(embed, weight, stop).to(device))

for seed, weight in zip(args.noise_prompt_seeds, args.noise_prompt_weights):
    gen = torch.Generator().manual_seed(seed)
    embed = torch.empty([1, perceptor.visual.output_dim]).normal_(generator=gen)
    pMs.append(Prompt(embed, weight).to(device))

try:
    if type(seed) is list:
        for s in seed:
            clear_output(wait=True)
            torch.manual_seed(s)
            print('Using seed:', s)
            run_seed()
            save_final(s)
    else:
        while batch_seed_count > 0: #unchecking run_batch just forces the count to be 1
            clear_output(wait=True)
            if run_batch or (args.seed is None) or (int(args.seed) < 0):
                s = torch.seed()
            else:
                s = args.seed
            torch.manual_seed(s)
            print('Using seed:', s)
            run_seed()
            save_final(s)
            batch_seed_count -= 1

except KeyboardInterrupt:
    del z
    del z_max
    del z_min
    del opt
    pass

In [ ]:
#@title Compress batch results

!mkdir /content/old_results
!zip -r /content/batch_results.zip {abs_root_path}/{args.hash}
!mv {abs_root_path}/{args.hash} /content/old_results/{args.hash}

!cp batch_results.zip /content/drive/MyDrive

In [ ]:
#@title Generate a video with the results
#@markdown Start and end points are percentages, default 5 and 100 just cuts off the gray noise at the beginning. Target length is in seconds.
#If you did a batch, this will generate a video only for the last image.

reset_value = i

start_point = 5 #@param {type:"slider", min:1, max:100, step:1}
end_point = 100 #@param {type:"slider", min:1, max:100, step:1}

init_frame = int(i*start_point/100)
last_frame = int(i*end_point/100)

min_fps = 10
max_fps = 30

total_frames = last_frame-init_frame

target_length = 15 #@param {type:"number"} #Video length in seconds.
length = target_length

frames = []
tqdm.write('Generating video...')
for i in range(init_frame,last_frame): #
    filename = f"steps/{i:04}.png"
    frames.append(Image.open(filename))

fps = np.clip(total_frames/length,min_fps,max_fps)

from subprocess import Popen, PIPE
p = Popen(['ffmpeg', '-y', '-f', 'image2pipe', '-vcodec', 'png', '-r', str(fps), '-i', '-', '-vcodec', 'libx264', '-r', str(fps), '-pix_fmt', 'yuv420p', '-crf', '17', '-preset', 'veryslow', 'video.mp4'], stdin=PIPE)
for im in tqdm(frames):
    im.save(p.stdin, 'PNG')
p.stdin.close()

print("The video is now being compressed, wait ... ")
p.wait()
print("The video is ready.")

i = reset_value

In [ ]:
# @title View video in browser
# @markdown This process may take a little longer. If you don't want to wait, download it by executing the next cell instead of using this cell.

mp4 = open('video.mp4','rb').read()
data_url = "data:video/mp4;base64," + b64encode(mp4).decode()
#Uncomment below line to show your prompt above the video.
print(prompts)
display.HTML("""
<video width=400 controls>
      <source src="%s" type="video/mp4">
</video>
""" % data_url)

In [ ]:
#@title Download video
#This is kind of dumb, since you can just right-click to download.
#Use this cell for something else.

from google.colab import files
files.download("video.mp4")

In [ ]:
#@title Optional CKPT file transfer
#@markdown Use this to make large files downloadable from Google Drive.

# Uncomment only the lines for the step you want to do. Do 1, then 2 to save the file(s). 
# Next time do 1, then 3 to copy the file(s) over instead of downloading.
# Change the filename to match the name of the model you want to save or load.

# 1. Mount your Google Drive.
#from google.colab import drive
#drive.mount('/content/drive')

# 2. Copy the CKPT file TO your Drive. (Do this for each model you want to save.)
#!cp wikiart_16384.ckpt /content/drive/MyDrive

# 3. Copy the CKPT file FROM your Drive.
#!cp /content/drive/MyDrive/ade20k.ckpt ade20k.ckpt
#!cp /content/drive/MyDrive/ffhq.ckpt ffhq.ckpt
#!cp /content/drive/MyDrive/celebahq.ckpt celebahq.ckpt
#!cp /content/drive/MyDrive/wikiart_1024.ckpt wikiart_1024.ckpt

# 4. Download only the YMAL file you need. (They're very small.)
# Just copy the line from the model selection step.
#!curl -L -o ade20k.yaml -C - 'https://static.miraheze.org/intercriaturaswiki/b/bf/Ade20k.txt' #ADE20K
#!curl -L -o ffhq.yaml -C - 'https://app.koofr.net/content/links/0fc005bf-3dca-4079-9d40-cdf38d42cd7a/files/get/2021-04-23T18-19-01-project.yaml?path=%2F2021-04-23T18-19-01_ffhq_transformer%2Fconfigs%2F2021-04-23T18-19-01-project.yaml&force' #FFHQ
#!curl -L -o celebahq.yaml -C - 'https://app.koofr.net/content/links/6dddf083-40c8-470a-9360-a9dab2a94e96/files/get/2021-04-23T18-11-19-project.yaml?path=%2F2021-04-23T18-11-19_celebahq_transformer%2Fconfigs%2F2021-04-23T18-11-19-project.yaml&force' #CelebA-HQ
#!curl -L -o wikiart_1024.yaml -C - 'http://dl.nmkd.de/ai/clip/wikiart-vqgan/WikiArt_augmented_Steps_7mil_finetuned_480k.yaml' #WikiArt 1024


# To prevent automatic disconnect due to inactivity:

Open inspector (ctrl+shift+i), click on console, and run this code in it (ctrl+enter):

```javascript
function ClickConnect() {
  console.log('Working')
  document
    .querySelector('#top-toolbar > colab-connect-button')
    .shadowRoot.querySelector('#connect')
    .click()
}
intervalTiming = setInterval(ClickConnect, 60000)
```

It could crash the browser!

If that doesn't work, try this:

```javascript
function ClickConnect(){
  console.log('Working');
  document.querySelector('colab-toolbar-button#toolbar-add-text').click();
}
intervalTiming = setInterval(ClickConnect, 60000);
```